## HPC Lab-9
Name: Tanvi Penumudy

Enroll No: E18CSE187

In [1]:
import multiprocessing
import time

In [2]:
import csv
from operator import itemgetter
import networkx as nx
from networkx.algorithms import community

In [3]:
# loading nodes
with open('lastfm_asia_target.csv', 'r') as nodecsv: 
    nodereader = csv.reader(nodecsv) 
    nodes = [n for n in nodereader][1:]

In [4]:
node_names = [n[0] for n in nodes]

In [5]:
# loading edges
with open('lastfm_asia_edges.csv', 'r') as edgecsv:
    edgereader = csv.reader(edgecsv) 
    edges = [tuple(e) for e in edgereader][1:]

In [6]:
print(len(node_names))
print(len(edges)) 

7624
27806


In [7]:
G = nx.Graph()

In [8]:
G.add_nodes_from(node_names) # importing nodes
G.add_edges_from(edges) # importing edges

In [9]:
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 7624
Number of edges: 27806
Average degree:   7.2943


In [10]:
print(type(node_names[0]))

<class 'str'>


In [11]:
# Selecting 16 from the Graph
print(list(nx.bfs_edges(G,'16')))

[('16', '7296'), ('16', '2626'), ('16', '131'), ('16', '6323'), ('16', '1799'), ('16', '3103'), ('16', '5578'), ('16', '7435'), ('16', '1677'), ('16', '5072'), ('16', '4008'), ('16', '3410'), ('16', '6446'), ('16', '4184'), ('16', '5725'), ('16', '1349'), ('16', '5088'), ('16', '6544'), ('16', '2530'), ('16', '1956'), ('16', '38'), ('16', '550'), ('16', '6567'), ('16', '4968'), ('16', '1005'), ('16', '6126'), ('16', '1905'), ('16', '3699'), ('16', '773'), ('16', '1016'), ('16', '2751'), ('16', '3324'), ('16', '766'), ('16', '6101'), ('7296', '1738'), ('7296', '1879'), ('7296', '2422'), ('7296', '2996'), ('7296', '4710'), ('7296', '4969'), ('7296', '5108'), ('7296', '5364'), ('7296', '6002'), ('7296', '6715'), ('7296', '6823'), ('2626', '33'), ('2626', '54'), ('2626', '187'), ('2626', '645'), ('2626', '1852'), ('2626', '1875'), ('2626', '1929'), ('2626', '2229'), ('2626', '2519'), ('2626', '5455'), ('2626', '6317'), ('2626', '5457'), ('2626', '6479'), ('2626', '7131'), ('2626', '2797'),

In [12]:
sample = list(nx.bfs_edges(G,'16'))
len(sample)

7623

In [13]:
type(nx.bfs_tree(G,'16'))

networkx.classes.digraph.DiGraph

In [14]:
tree = nx.bfs_tree(G,'16')

In [27]:
l = list(nx.all_pairs_shortest_path_length(tree))

In [28]:
len(l)

7624

In [52]:
# Serial (For 200 Nodes)
%%time
t = time.time()
def worker1(nodes, edges): 
  G = nx.Graph()
  G.add_nodes_from(nodes)
  G.add_edges_from(edges)
  for i in nodes:
    nx.all_pairs_shortest_path_length(nx.bfs_tree(G,i))
  print("Done 1")

def worker2(nodes, edges):
  G = nx.Graph()
  G.add_nodes_from(nodes)
  G.add_edges_from(edges)
  for i in nodes:
    nx.all_pairs_shortest_path_length(nx.bfs_tree(G,i))
  print("Done 2")

worker1(node_names[:100],edges)
worker2(node_names[-100:],edges)
print("Done!")

print(time.time()-t)

Done 1
Done 2
Done!
18.567119359970093
CPU times: user 18.5 s, sys: 102 ms, total: 18.6 s
Wall time: 18.6 s


In [46]:
# Parallel (For 200 Nodes)
%%time
t = time.time()
def worker1(nodes, edges): 
  G = nx.Graph()
  G.add_nodes_from(nodes)
  G.add_edges_from(edges)
  for i in nodes:
    nx.all_pairs_shortest_path_length(nx.bfs_tree(G,i))
  print("Done 1")

def worker2(nodes, edges):
  G = nx.Graph()
  G.add_nodes_from(nodes)
  G.add_edges_from(edges)
  for i in nodes:
    nx.all_pairs_shortest_path_length(nx.bfs_tree(G,i))
  print("Done 2")

if __name__ == "__main__": 

  # creating processes 
  p1 = multiprocessing.Process(target=worker1) 
  p2 = multiprocessing.Process(target=worker2)

  # starting processes 
  p1.start() 
  p2.start()

  # wait until processes are finished 
  p1.join() 
  p2.join()

  worker1(node_names[:100],edges)
  worker2(node_names[-100:],edges)
  print("Done!")

print(time.time()-t)

Process Process-9:
Traceback (most recent call last):
Process Process-10:
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
TypeError: worker1() missing 2 required positional arguments: 'nodes' and 'edges'
TypeError: worker2() missing 2 required positional arguments: 'nodes' and 'edges'


Done 1
Done 2
Done!
18.233521699905396
CPU times: user 18.1 s, sys: 95.2 ms, total: 18.2 s
Wall time: 18.2 s
